In [1]:
import cv2

# 동영상 파일 경로 설정
video_path = './highway01.mp4'

# 동영상 파일 읽기
cap = cv2.VideoCapture(video_path)

In [2]:
# Background Subtractor 설정 (MOG2)
fgbg = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=40, detectShadows=True)

while True:
    # 프레임 읽기
    ret, frame = cap.read()
    
    if not ret:
        break
    
    # 배경 차분 적용
    fgmask = fgbg.apply(frame)
    
    # 모폴로지 연산을 통해 노이즈 제거
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    
    # 윤곽선 찾기
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        # 작은 노이즈 제거
        if cv2.contourArea(contour) > 500:
            # 객체의 경계 상자 그리기
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    # 결과 화면 표시
    cv2.imshow('Frame', frame)
    cv2.imshow('Foreground Mask', fgmask)
    
    # ESC 키를 누르면 종료
    if cv2.waitKey(30) & 0xFF == 27:
        break

# 동영상 파일 해제
cap.release()
cv2.destroyAllWindows()